In [1]:
import osmnx as ox
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import folium
from geopy.distance import distance
from geopy import Point
import networkx as nx
from shapely.geometry import Point, LineString
import numpy as np

In [2]:
#find the street networks based on location name
G = ox.graph_from_place('Gili Trawangan', network_type='walk') 
streets_graph = ox.project_graph(G)
#save only nodes to geodataframe
gdf_points = ox.graph_to_gdfs(streets_graph, nodes=True, edges=False,node_geometry=False, fill_edge_geometry=True)


In [3]:
#determine your starting point
start = ( -8.348122,116.038095) 
#replace the value of start point with the nearest node to the original start point
start_node = ox.get_nearest_node(G, start) 
# Calculate the shortest path from start point to each node within streets network
gdf_points['route_to_start_point'] = gdf_points.apply(lambda row: nx.shortest_path(G, start_node, ox.get_nearest_node(G, (row['lat'],row['lon'])), weight='length'), axis=1)

In [4]:
gdf = []
for x in range(gdf_points.shape[0]):
    #find all nodes in the same route/path
    data = gdf_points.loc[gdf_points['route_to_start_point'].values[x]]
    data['prev_lon'] = data['lon'].shift(1)
    data.loc[data['prev_lon'].isnull() == True, 'prev_lon'] = data['lon']
    data['prev_lat'] = data['lat'].shift(1)
    data.loc[data['prev_lat'].isnull() == True, 'prev_lat'] = data['lat']
    #calculate distance from one point to the next one
    data['distance_from_prev_point'] = data.apply(lambda row: distance((row['lat'],row['lon']), (row['prev_lat'],row['prev_lon'])).meters,axis=1)
    data['distance_cumulative'] = data['distance_from_prev_point'].cumsum()
    data['id'] = x+1
    data = data.reset_index()
    data['no_order'] = data.index
    data['distance_cumulative_prev_point'] = data['distance_cumulative'] - data['distance_from_prev_point']
    # Zip the coordinates into a point object and convert to a GeoDataFrame
    geometry = data.apply(lambda row: LineString([Point(row['prev_lon'], row['prev_lat']), Point(row['lon'], row['lat'])]),axis=1)
    data = GeoDataFrame(data, geometry=geometry)
    #Create a new points at the specified distance along a linear geometric object. This only applied to geometric objects which have cumulative distance more than 800 & 1000, and the cumulative distance of previous objects are below 800 & 1000
    data.loc[(data['distance_cumulative'] > 800) & (data['distance_cumulative_prev_point'] < 800), 'interpolation'] = (800-(data['distance_cumulative']-data['distance_from_prev_point']))/data['distance_from_prev_point']
    data.loc[(data['distance_cumulative'] > 1000) & (data['distance_cumulative_prev_point'] < 1000), 'interpolation'] = (1000-(data['distance_cumulative']-data['distance_from_prev_point']))/data['distance_from_prev_point']
    data.loc[data['interpolation'].isnull() == False, 'new_point'] = data.apply(lambda row: row['geometry'].interpolate(row['interpolation'], normalized=True).wkt, axis=1)
    #create new dataframe for the new point/s
    df_new_point = data[data['interpolation'].isnull() == False]
    if (df_new_point.shape[0] > 0):
        #change the longitude and latitude columns with the new point/s
        df_new_point['lon']  = df_new_point['new_point'].str.strip('POINT (').str.strip(')').str.split(' ').str[0] 
        df_new_point['lat']  = df_new_point['new_point'].str.strip('POINT (').str.strip(')').str.split(' ').str[1]
        #recalculate the distance along a linear geometric object
        df_new_point['distance_from_prev_point'] = df_new_point.apply(lambda row: distance((row['lat'],row['lon']), (row['prev_lat'],row['prev_lon'])).meters,axis=1)
        df_new_point['distance_cumulative']  = 0
    #merge both  dataframes
    df_full = pd.concat([df_new_point,data], axis=0)
    df_full = df_full.sort_values(by=['no_order','distance_from_prev_point']).reset_index()
    #since we have new dataframe, we need to recalculate the distance from one point to the next one
    df_full['prev_lon'] = df_full['lon'].shift(1)
    df_full.loc[df_full['prev_lon'].isnull() == True, 'prev_lon'] = df_full['lon']
    df_full['prev_lat'] = df_full['lat'].shift(1)
    df_full.loc[df_full['prev_lat'].isnull() == True, 'prev_lat'] = df_full['lat']
    df_full['distance_from_prev_point'] = df_full.apply(lambda row: distance((row['lat'],row['lon']), (row['prev_lat'],row['prev_lon'])).meters,axis=1)
    df_full['distance_cumulative'] = df_full['distance_from_prev_point'].cumsum()
    #remove records with less than 0 m distance
    df_full = df_full[df_full['distance_from_prev_point'] > 0]
    
    
    
    gdf.append(df_full)
    
    
        
        
gdf = pd.concat(gdf)   

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [5]:
gdf = gdf.drop_duplicates(subset=["lon","lat","prev_lon","prev_lat"], keep="last")

In [6]:
gdf['prev_lat'] = gdf['prev_lat'].astype(float)
gdf['prev_lon'] = gdf['prev_lon'].astype(float)
gdf['lat'] = gdf['lat'].astype(float)
gdf['lon'] = gdf['lon'].astype(float)

In [7]:
# Zip the coordinates into a point object and convert to a GeoDataFrame
geometry = gdf.apply(lambda row: LineString([Point(row['prev_lon'], row['prev_lat']), Point(row['lon'], row['lat'])]),axis=1)
gdf = GeoDataFrame(gdf, geometry=geometry)


In [8]:
gdf.crs = {'init' :'epsg:4326'}
m = folium.Map([-8.348122,116.038095],
zoom_start=15,
tiles="CartoDb dark_matter")

# remove record/s with less than 1200 m from centre point
gdf_routes = gdf[gdf['distance_cumulative']<1200]


# Display routes within 800m (green), 1000m (yellow), and 1200m (red) extension from the start point.
folium.GeoJson(gdf_routes[(gdf_routes['distance_cumulative']<1201)][['distance_cumulative','geometry']], style_function=lambda x: {'color': '#e6053a', 'weight':'1'}).add_to(m)
folium.GeoJson(gdf_routes[gdf_routes['distance_cumulative']<1001][['distance_cumulative','geometry']], style_function=lambda x: {'color': '#e6e31d', 'weight':'1'}).add_to(m)
folium.GeoJson(gdf_routes[gdf_routes['distance_cumulative']<801][['distance_cumulative','geometry']], style_function=lambda x: {'color': '#309b3b', 'weight':'1'}).add_to(m)
folium.CircleMarker(location=(-8.348122,116.038095), color = "#F4A6F7", radius=4).add_to(m)

locs = zip(gdf_routes['lat'], gdf_routes['lon'])
for location in locs:
    folium.CircleMarker(location=location, 
        color = "#F4F6F7",   radius=0.01).add_to(m)

# for i in range(0,len(gdf_routes)):
#     folium.Marker(location=(gdf_routes.iloc[i]['lat'], gdf_routes.iloc[i]['lon']), 
#         color = "#F4F6F7",  popup=gdf_routes.iloc[i]['distance_cumulative'] ,radius=0.01).add_to(m)

m

/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
